In [1]:
% Robert Fortunato
% Define parameters
num_bits = 1e6;  % Number of bits
power = 10^(-7);  % Received power
voltage = sqrt(power);  % Voltage representation
amplitude = voltage;  % Transmitted signal amplitude

snr_db_values = 0:2:18;  % SNR in dB (steps of 2)
channel_coefficients = [1, 0.1, 0.01, 0.001];  % Defined channel coefficients
ber_results = zeros(length(channel_coefficients), length(snr_db_values));  % Store BER results

% Loop over different channel coefficients
for c = 1:length(channel_coefficients)
    h = channel_coefficients(c);

    % Loop over different SNR values
    for i = 1:length(snr_db_values)
        snr_db = snr_db_values(i);
        snr_linear = 10^(snr_db / 10);  % Convert SNR from dB to linear
        noise_var = power / snr_linear;  % Calculate noise variance

        % Generate random bits (0s and 1s)
        bits = randi([0, 1], 1, num_bits);

        % Transmit signal (0 -> amplitude, 1 -> -amplitude)
        transmitted_signal = amplitude * (1 - 2 * bits);

        % Apply channel coefficient
        if h == 0
            % No signal passes through the channel; BER is 1
            ber_results(c, i) = 1.0;
            continue;
        end

        received_signal = h * transmitted_signal;  % Apply channel effect

        % Generate complex Gaussian noise
        noise_real = (1 / sqrt(2)) * sqrt(noise_var) * randn(1, num_bits);
        noise_imag = (1 / sqrt(2)) * sqrt(noise_var) * randn(1, num_bits);
        noise = noise_real + 1j * noise_imag;  % Complex Gaussian noise

        % Add noise to received signal
        received_signal = received_signal + noise;

        % Equalization: Avoid division by 0
        received_signal = received_signal / (h + 1e-10);  % Small value added to prevent divide by zero error

        % Convert received signal to real values
        received_signal_real = real(received_signal);

        % Decode bits: If received value < 0, it's a 1, else it's a 0
        decoded_bits = received_signal_real < 0;

        % Calculate Bit Error Rate (BER)
        bit_errors = sum(decoded_bits ~= bits);
        ber = bit_errors / num_bits;
        ber_results(c, i) = ber;
    end
end

% Plot BER vs. SNR for different channel coefficients
figure;
hold on;
plot_styles = {'o-', 's-', 'd-', '^-'};  % Different markers for each coefficient

for c = 1:length(channel_coefficients)
    semilogy(snr_db_values, ber_results(c, :), plot_styles{c}, 'LineWidth', 2, ...
             'DisplayName', sprintf('Channel Coeff = %.4f', channel_coefficients(c)));
end

xlabel('SNR (dB)');
ylabel('Bit Error Rate (BER)');
title('BER vs. SNR for Different Channel Coefficients');
grid on;
legend show;

% Ensure y-axis is log scale
set(gca, 'YScale', 'log');

hold off;

SyntaxError: invalid syntax (<ipython-input-1-2174a35c6ae0>, line 3)